In [1]:
import pandas as pd

df = pd.read_csv("text.csv", encoding="utf-8-sig")
# df.set_index(["时间", "股票代码"], inplace=True)
df.head(5)

,序号,股票代码,股票代码（两市）,文章标题,时间,文章详情,情绪打分,情绪打分2,一致=0
0,1,62689,000861.SZ,海印股份称今珠多糖注射液专利尚未取得 不存在炒作股价,2019/6/22,6月22日凌晨，广东海印集团股份有限公司(<em>000861</em>，以下简称“海印...,1,1.0,0
1,2,109423,002173.SZ,创新医疗：信披违规 遭浙江证监局警示,2019/1/31,创新医疗(<em>002173</em>)1月31日晚公告，公司当日收到浙江证监局下达...,0,0.0,0
2,3,185363,002639.SZ,雪人股份：控股股东将转让5%股份转让给福建省纾困一号股权投资合伙企业,2019/12/19,雪人股份12月19日晚间公告，控股股东林汝捷与福建省纾困一号股权投资合伙企业于2019...,1,1.0,0
3,4,131933,002322.SZ,理工环科：2021年半年度土壤修复营业收入为416万元 占营业收入比重为0.95%,2021/11/11,每经AI快讯，有投资者在投资者互动平台提问：你好董秘，公司在土壤修复方面营业收入大致占多...,2,1.0,1
4,5,396704,600375.SH,华菱星马：吉利商用车集团收购公司股权案通过反垄断审查,2020/9/1,9 月 1日，华菱星马发布公告称，2020年9月1日，公司收到吉利商用车集团转来的国家...,2,2.0,0


In [2]:
import numpy as np
import re
import jieba
from gensim.models import Word2Vec

def preprocess(ar):
    for i, con in enumerate(ar):
        # 去掉标点或其他特殊符号
        con = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+：~", "", con)
        # jieba分词
        ar[i] = jieba.lcut(con, cut_all=False)
    return ar


def train_w2v_model(ar):
    model = Word2Vec(ar.flatten(), workers=4)
    return model

def word2vec(ar, model):
    para_avg_vec = []
    # print("training finished")
    # 对每一段
    for para in ar:
        sum_vec = np.zeros(100)
        # 对每一段中每一个词
        for word in para:
            try:
                # 试着从模型中取得word vector
                sum_vec += model.wv[word]
            except(KeyError):
                # 如果有未记录的特殊符号，就跳过
                continue
        # 对词向量求平均得到句子的向量表示
        para_avg_vec.append(sum_vec / len(para))
    return para_avg_vec

In [3]:
#word_2_vec_model = train_w2v_model(np.array(df["文章详情"]))
#word_2_vec_model.save('w2v_model')
word_2_vec_model = Word2Vec.load('w2v_model')
vec = word2vec(np.array(df["文章详情"]), word_2_vec_model)

In [4]:
df["vec"] = vec
print(df["文章详情"][0])
df["vec"][0]

　　6月22日凌晨，广东海印集团股份有限公司(<em>000861</em>，以下简称“海印股份”)发布公告回复了日前深交所对该公司将参与“非洲猪瘟疫苗”投产一事下发的关注函。海印股份在公告中称，许启太团队研制的“今珠多糖注射液”属中药与天然药物制剂，并非疫苗。专利正在申请，尚未正式取得。　　此前...


array([ 0.09405213,  0.00813369, -0.41396803,  0.12234212, -0.09115982,
        0.27925893,  0.27831387,  0.04999109,  0.5007385 , -0.1825799 ,
        0.66568163,  0.2137591 , -0.00985369,  0.27510145, -0.10254665,
        0.00870984,  0.41533407,  0.06302615,  0.18787246,  0.0217877 ,
        0.16150123, -0.4840593 ,  0.29276585,  0.10296157, -0.26743265,
        0.01934119,  0.07534989, -0.03224325,  0.09248794, -0.09647534,
       -0.17463405, -0.27878453,  0.21552915, -0.14428115,  0.19215189,
       -0.34604797, -0.27785272, -0.1105668 , -0.21984906,  0.54569034,
        0.22501971, -0.29924741,  0.10160562,  0.10169786, -0.13170739,
       -0.30589651, -0.03016745, -0.17559056,  0.14371196,  0.30427059,
        0.06986884,  0.1876137 , -0.12997348,  0.6415827 , -0.0997327 ,
        0.18650104,  0.04861585, -0.31203435,  0.25392477, -0.15536632,
        0.05697248, -0.35699273,  0.29066136,  0.35757723, -0.06944587,
       -0.07997819, -0.11174433,  0.18157676, -0.10593209,  0.17

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df["vec"], df["情绪打分"], test_size=0.2, random_state=16) # 今日几号？16号咯

In [6]:
from tensorflow.keras import layers, losses
import tensorflow as tf

x_train, x_test = tf.convert_to_tensor(x_train.to_list()), tf.convert_to_tensor(x_test.to_list())
y_train, y_test = tf.convert_to_tensor(y_train), tf.convert_to_tensor(y_test)

In [7]:
def create_model(num_labels):
  model = tf.keras.Sequential([
      layers.Dense(1024, activation="swish"),
      layers.Dense(1024, activation="swish"),
      layers.Dense(128, activation="swish"),
      layers.Dense(num_labels)
  ])
  return model


model = create_model(num_labels=1)

model.compile(
    loss = "mae",
    optimizer='adam',
)

history = model.fit(
    x_train, y_train, epochs=100, validation_data=(x_test, y_test)
)
model.save('dnn')

Epoch 1/100
100/100 [==============================] - 2s 18ms/step - loss: 0.6432 - val_loss: 0.5461
Epoch 2/100
100/100 [==============================] - 2s 17ms/step - loss: 0.5760 - val_loss: 0.5423
Epoch 3/100
100/100 [==============================] - 2s 16ms/step - loss: 0.5689 - val_loss: 0.5520
Epoch 4/100
100/100 [==============================] - 2s 17ms/step - loss: 0.5628 - val_loss: 0.5459
Epoch 5/100
100/100 [==============================] - 2s 17ms/step - loss: 0.5374 - val_loss: 0.5219
Epoch 6/100
100/100 [==============================] - 2s 17ms/step - loss: 0.5325 - val_loss: 0.5048
Epoch 7/100
100/100 [==============================] - 2s 17ms/step - loss: 0.5224 - val_loss: 0.4918
Epoch 8/100
100/100 [==============================] - 2s 17ms/step - loss: 0.5137 - val_loss: 0.4993
Epoch 9/100
100/100 [==============================] - 2s 17ms/step - loss: 0.5125 - val_loss: 0.4938
Epoch 10/100
100/100 [==============================] - 2s 17ms/step - loss: 0.510

INFO:tensorflow:Assets written to: dnn\assets


INFO:tensorflow:Assets written to: dnn\assets


In [8]:
def accuracy(pred, y, num=[0, 2], num_labels=3):
    delta = num[1] - num[0]
    d = delta / num_labels
    acc = 0
    for i in range(len(pred)):
        c = 0
        if pred[i] <= num[0] + d:
            pred[i] = c
        elif pred[i] <= num[0] + 2*d:
            pred[i] = c + 1
        else:
            pred[i] = c + 2
    for i in range(len(pred)):
        if pred[i] == y[i]:
            acc += 1
    return acc / len(pred)

In [9]:
pred = model.predict(x_test)
accuracy(pred, y_test)

25/25 [==============================] - 1s 13ms/step


0.62875